In [5]:
import numpy as np
import scipy as sp
import suite2p as s2p
import dill

import TwoPUtils as tpu
import STX3KO_analyses as stx
import os
from matplotlib import pyplot as plt


%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = 'mCherry8'

file_list = stx.ymaze_sess_deets.CTRL_sessions[mouse]
for f in file_list:
    print(f)


{'date': '14_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 0}
{'date': '15_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 1}
{'date': '16_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 2}
{'date': '17_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 3}
{'date': '18_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 4}
{'date': '20_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 5}
({'date': '21_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 1, 'novel_arm': 1, 'ravel_ind': 6}, {'date': '21_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 7})
{'date': '22_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 8}
({'date': '22_10_2021', 'scene': 'YM

In [4]:
def check_and_make_h5s(file_list, runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        source_folder  = os.path.join('/mnt','BigDisk','2P_scratch')
        source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
        info = tpu.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ =  tpu.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=False)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [6]:
def run_session(fdict):
    _h5list,_frs = check_and_make_h5s([fdict,],runh5=True)
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')

    ops = tpu.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 


for f in file_list[6:]: #file_list:
    
    if isinstance(f,tuple):
        for _f in f:
            run_session(_f)
    else:
        run_session(f)
    

{'date': '21_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 1, 'novel_arm': 1, 'ravel_ind': 6}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 8954
['/mnt/BigDisk/2P_scratch/mCherry8/21_10_2021/YMaze_LNovel/YMaze_LNovel_001_001']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/mCherry8/21_10_2021/YMaze_LNovel/YMaze_LNovel_001_001/YMaze_LNovel_001_001.h5']
time 452.79 sec. Wrote 8954 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 8954 frames
Reference frame, 16.38 sec.
Registered 2000/8954 in 109.43s
Registered 4000/8954 in 203.88s
Registered 6000/8954 in 299.40s
Registered 8000/8954 in 393.14s
Registered second channel in 220.00 sec.
----------- Total 677.52 sec
Registration metrics, 46.53 sec.
NOTE:

In [1]:
def update_sess_dict(mouse, f):
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])

            
    source_folder = "/mnt/BigDisk/2P_scratch"
    source_stem = os.path.join(source_folder, mouse, f['date'], f['scene'], scan_str)

    f['mouse']=mouse
    f.update({'scan_file': source_stem + '.sbx',
              'scanheader_file': source_stem + '.mat',
              'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
              'scan_number': f['scan'],
              'prompt_for_keys': False,
              'VR_only': False,
              'scanner': "NLW",
             })
    return f

def run_and_save(fdict,comb_frames=0):
#     with open(os.path.join('/home/mplitt/YMazeSessPkls',mouse,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
#         sess = dill.load(file)
#     sess.novel_arm = fdict['novel']    
            
    sess = stx.session.YMazeSession(**fdict)
    sess.load_scan_info()
    sess.align_VR_to_2P()
    sess.load_suite2p_data(which_ts=('F', 'Fneu','spks'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
    sess.get_trial_info()
    sess.neuropil_corrected_dff()
    
    
    sess.place_cells_calc(nperms=1000)
    
    sess.add_pos_binned_trial_matrix('F_dff')
    sess.add_timeseries(F_dff_norm = sess.timeseries['F_dff']/np.nanpercentile(sess.timeseries['F_dff'],99,axis=1,keepdims=True))
    sess.rm_pos_binned_trial_matrix('F_dff_norm')
    sess.add_pos_binned_trial_matrix('F_dff_norm')
    
    sess.add_pos_binned_trial_matrix('spks') #,speed=sess.vr_data['dz']._values, speed_thr = .05)  
    sess.add_timeseries(spks_norm = sess.timeseries['spks']/np.nanpercentile(sess.timeseries['spks'],99,axis=1,keepdims=True))
    sess.add_pos_binned_trial_matrix('spks_norm') 

    sess.add_timeseries(licks=sess.vr_data['lick']._values)
    sess.add_pos_binned_trial_matrix('licks')
    sess.add_timeseries(speed=sess.vr_data['dz']._values)
    sess.add_pos_binned_trial_matrix('speed')
    
    tpu.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')
    return comb_frames+sess.scan_info['max_idx']

In [4]:
for f in file_list: #file_list:
    print(f)
    if isinstance(f,tuple):
        for _f in f:
            f_ = update_sess_dict(mouse,_f)
            _ = run_and_save(f_)
    else:
        f_ = update_sess_dict(mouse,f) 
        _ = run_and_save(f_)

{'date': '14_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 0}
Fixing teleports
(150891,) (31047,)
2002.087153652393 2001.9586901763223


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


/mnt/BigDisk/2P_scratch/mCherry8/14_10_2021/YMaze_LNovel/YMaze_LNovel_001_003/suite2p
233 4108
5045 8440
9379 13575
14516 18957
19926 23699
24639 31046
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '15_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 1}
Fixing teleports
(153616,) (31551,)
2034.5692695214107 2034.4584382871535
/mnt/BigDisk/2P_scratch/mCherry8/15_10_2021/YMaze_LNovel/YMaze_LNovel_001_003/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


197 4044
4987 8581
9523 13276
14214 18425
19366 23061
24002 31550
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '16_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 2}
Fixing teleports
(168801,) (34740,)
2240.173425692695 2240.096725440806
/mnt/BigDisk/2P_scratch/mCherry8/16_10_2021/YMaze_LNovel/YMaze_LNovel_001_002/suite2p
185 4812
5753 9921
10869 15090
16032 20286
21231 25547
26487 34739
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
{'date': '17_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3, 'novel_arm': 1, 'ravel_ind': 3}
Fixing teleports
(167780,) (34602,)
2231.315491183879 2231.19798488665


...

/mnt/BigDisk/2P_scratch/mCherry8/22_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_001_002/suite2p
171 5024
5958 10038
10973 14757
15691 19708
20650 24223
25158 32546
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
({'date': '22_10_2021', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 2, 'novel_arm': 1, 'ravel_ind': 9}, {'date': '22_10_2021', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4, 'novel_arm': 1, 'ravel_ind': 10})
Fixing teleports
(43416,) (9834,)
634.2098236775819 634.0675062972292


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


/mnt/BigDisk/2P_scratch/mCherry8/22_10_2021/YMaze_RewardReversal/YMaze_RewardReversal_002_002/suite2p
176 4638
5574 9833
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
Fixing teleports
(131906,) (26631,)
1717.3056675062971 1717.1989924433249
/mnt/BigDisk/2P_scratch/mCherry8/22_10_2021/YMaze_LNovel/YMaze_LNovel_001_004/suite2p


/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:233: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:239: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


207 4921
5858 10737
11673 16464
17400 26630
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900
perm 0
perm 100
perm 200
perm 300
perm 400
perm 500
perm 600
perm 700
perm 800
perm 900


In [3]:
def single_mouse_aligner(mouse, sessions):
    sess_dir = os.path.join('/home/mplitt/YMazeSessPkls',mouse)
    sess_list = []
    for sess_deets in sessions:     
        if isinstance(sess_deets,(list,tuple)):
            for _sess_deets in sess_deets:
                with open(os.path.join(sess_dir,_sess_deets['date'],"%s_%d.pkl" % (_sess_deets['scene'], _sess_deets['session'])),'rb') as file:
                    sess_list.append(dill.load(file))
        else:
            with open(os.path.join(sess_dir,sess_deets['date'],"%s_%d.pkl" % (sess_deets['scene'], sess_deets['session'])),'rb') as file:
                    sess_list.append(dill.load(file))

    sa = tpu.roi_matching.ROIAligner(sess_list)
    sa.run_pairwise_matches()

    with open(os.path.join(sess_dir,'roi_aligner_results.pkl'), 'wb') as file:
        dill.dump(sa.match_inds,file)

In [6]:
mouse = 'mCherry8'
sessions = stx.ymaze_sess_deets.CTRL_sessions[mouse]
single_mouse_aligner(mouse,sessions)

thresh 0.2819738516807556
thresh 0.28012259101867676
thresh 0.3390484879016876
thresh 0.33511285281181336
thresh 0.2900409529209137
thresh 0.2916462848186493
thresh 0.3266937861442566
thresh 0.3062133321762085
thresh 0.3188672194480896
thresh 0.31568677520751953
thresh 0.3137567768096924
thresh 0.3523081967830658
thresh 0.35364915227890015
thresh 0.34445728158950806
thresh 0.31665433740615845
thresh 0.3229105005264282
thresh 0.3596988151073456
thresh 0.36095835065841675
thresh 0.2931763062477112
thresh 0.3028958866596222
thresh 0.3626892695426941
thresh 0.30884615874290466
thresh 0.36874078011512756
thresh 0.34190474247932434
thresh 0.3758507797718048
thresh 0.3236681351661682
thresh 0.34242160415649414
thresh 0.3551789948940277
thresh 0.3593906292915344
thresh 0.307995689868927
thresh 0.33410121297836304
thresh 0.3231852481365204
thresh 0.3243357071876526
thresh 0.3459079394340515
thresh 0.3388429114818573
thresh 0.3746972212791443
thresh 0.33103461360931397
thresh 0.35935301876068115